In [1]:
import shutil
import os

In [2]:
pwd = os.getcwd()
#pwd += "/rheo_bench/channel/"
pwd

'/qscratch/etpaler/rheo_benchmark_repository/benchmark_3__poiseuille_flow_newtonian'

# Generate LAMMPS Input Scripts

In [3]:
#parameter grids
# var_model_grid = {"fix                2 all rheo/viscosity constant ${eta}":"0",
#                   "fix                2 all rheo/viscosity power ${eta} ${gd0} ${K} ${npow}":"1"}
var_model_grid = {"fix                2 all rheo/viscosity constant ${eta}":"0"}
var_kernel_grid = {"CRK0":"0","CRK1":"1","CRK2":"2","Quintic":"5"}
var_sf_grid = {"0.5":"2","0.25":"4","0.2":"5","0.1":"10","0.05":"20"}
var_hd_grid = {"3.0":"3p0"}
var_eta_grid = {"0.16":"p16"}
var_fext_grid = {"1e-4":"1e-4"}


In [4]:
#Generate lammps files

out_filepath = "data" #don't use trailing slash

filenames = []
for model in var_model_grid:
    for kernel in var_kernel_grid:
        for sf in var_sf_grid:
            for eta in var_eta_grid:
                for fext in var_fext_grid:
        
                    filename = var_model_grid[model]+"_"+var_kernel_grid[kernel]+"_"+var_sf_grid[sf]+"_"+var_eta_grid[eta]+"_"+var_fext_grid[fext]
                    file = shutil.copyfile("in.template.lmp","in."+filename+".lmp")
                    file_in = open(file,"rt")
                    file_text = file_in.read()

                    #Replace variables
                    file_text = file_text.replace("var_sf",sf)
                    file_text = file_text.replace("var_fext",fext)
                    file_text = file_text.replace("var_eta",eta)
                    file_text = file_text.replace("var_kernel",kernel)
                    file_text = file_text.replace("var_model",model)

                    file_text = file_text.replace("var_out_filepath",out_filepath)
                    file_text = file_text.replace("var_filename",filename)

                    file_out = open(file,"wt")
                    file_out.write(file_text)
                    
                    filenames.append(filename)
                    file_out.close()

In [5]:
filenames

['0_0_2_p16_1e-4',
 '0_0_4_p16_1e-4',
 '0_0_5_p16_1e-4',
 '0_0_10_p16_1e-4',
 '0_0_20_p16_1e-4',
 '0_1_2_p16_1e-4',
 '0_1_4_p16_1e-4',
 '0_1_5_p16_1e-4',
 '0_1_10_p16_1e-4',
 '0_1_20_p16_1e-4',
 '0_2_2_p16_1e-4',
 '0_2_4_p16_1e-4',
 '0_2_5_p16_1e-4',
 '0_2_10_p16_1e-4',
 '0_2_20_p16_1e-4',
 '0_5_2_p16_1e-4',
 '0_5_4_p16_1e-4',
 '0_5_5_p16_1e-4',
 '0_5_10_p16_1e-4',
 '0_5_20_p16_1e-4']

# Write the Solo Jobscript

In [6]:
account="FY150028"
job_name="Newt_channel"
time="48:0:0"
partition="batch"
nodes="4"
ntasks_per_node="16"
run_command = "mpirun -np 16 /ascldap/users/etpaler/dev/rlammps_extrusion_general_cs2/build/lmp -in "

In [7]:
with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        #---slurm commands---#
        jobscript.write("#SBATCH --account={}\n".format(account))
        jobscript.write("#SBATCH --job-name={}\n".format(job_name))
        jobscript.write("#SBATCH --time={}\n".format(time))
        jobscript.write("#SBATCH --partition={}\n".format(partition))
        jobscript.write("#SBATCH --nodes={}\n".format(nodes))
        jobscript.write("#SBATCH --ntasks-per-node={}\n\n".format(ntasks_per_node))
        
         #---unix commands---#
        jobscript.write("module load cmake/3.20.3 intel/18.0.0.128 openmpi-intel/4.1 mkl/18.0.0.128\n\n")
        
        for filename in filenames:
            jobscript.write(run_command+"in."+filename+".lmp\n")
            
        jobscript.write("\n")

# Write the WSL Jobscript

In [24]:
run_command = "mpirun -np 4 ~/dev/rlammps_wall_debug/build/lmp -in "

with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")